# *Unsupervised learning: k-means clustering*

[http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [ ]:
## Importing basic packages

import os
import random
import numpy as np

In [ ]:
%%capture
## Downloading several hundred New York Times articles as text files

os.chdir('/sharedfolder/')

!wget -N https://github.com/pcda17/pcda17.github.io/raw/master/week/10/nyt_articles_11-9-2017.zip
!unzip -o nyt_articles_11-9-2017.zip

In [ ]:
## Loading all text files in the current directory as a list of strings

os.chdir('/sharedfolder/nyt_articles_11-9-2017/')

document_list = []

for filename in [item for item in os.listdir('./') if '.txt' in item]: # Excluding files other than .txt with a list comprehension
    text_data = open(filename).read()
    document_list.append(text_data)

print(len(document_list)) # Printing number of documents in list

random.choice(document_list) # Viewing a single document chosen at random

In [ ]:
## Creating stop word list

from nltk.corpus import stopwords
import string

stop_words = stopwords.words('english') + ["'s", "'re", '”', '“', '‘', '’', '—'] + list(string.punctuation)

stop_words[:10]     ## Viewing first 10 stop words

In [ ]:
## Tokenizing, stemming, and removing stop words from our list of documents

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

documents_filtered = []

for document in document_list:
    token_list = word_tokenize(document.lower())                                 ## Tokenizing
    tokens_filtered = [item for item in token_list if (item not in stop_words)]  ## Removing stop words
    tokens_filtered = [stemmer.stem(item) for item in tokens_filtered]           ## Stemming
    documents_filtered.append(' '.join(tokens_filtered))

In [ ]:
## Viewing a preprocessed document

random.choice(documents_filtered)

In [ ]:
## Vectorizing preprocessed documents

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(documents_filtered) 

In [ ]:
## Creating a vocabulary list corresponding to the vectors we created above

vocabulary = vectorizer.get_feature_names()

vocabulary[1140:1160]

In [ ]:
## Clustering documents

from sklearn.cluster import KMeans

kmeans_classifier = KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=500, algorithm='auto')
                                   ## ^ docuements to be grouped in 8 clusters 

cluster_classes = kmeans_classifier.fit_predict(X) 

cluster_classes[:20]               ## Viewing first 20 cluster assignments

In [ ]:
## Our cluster assignments and document lists are the same size, in the same order.

print(len(cluster_classes))

print(len(documents_filtered))

print(len(document_list))

In [ ]:
## We can view a document and its assigned cluster by accessing the same index in both lists

index_num = 130

print('Cluster assignment:')
print(cluster_classes[index_num])
print()
print('Document:')
print(document_list[index_num])

In [ ]:
## Write each document to a new text file, with each cluster of documents in its own directory

try: os.mkdir('/sharedfolder/nyt_clusters_11-9-2017/')
except: pass

for i in range(len(documents_filtered)):
    
    out_dir = '/sharedfolder/nyt_clusters_11-9-2017/Cluster_' + str(cluster_classes[i])  ## Creating a directory pathname that
                                                                                         ## includes the assigned cluster number.
    try: os.mkdir(out_dir)  ## Creating the out_dir directory if it does not yet exist
    except: pass
    
    os.chdir(out_dir)
    
    out_filename = 'Document_' + str(i) + '.txt'    ## Creating a filename for the text file
    
    with open(out_filename, 'w') as file_out:
        file_out.write(document_list[i])            ## Writing text from original (non-preprocessed) document list


In [ ]:
## Classifying a new text into an existing cluster

input_vector = vectorizer.transform(['Even the budget office is revising its estimates and has predicted the new numbers would be smaller.'])

kmeans_classifier.predict(input_vector)

## *Scatter Plots*

In [ ]:
## A simple scatter plot example

import random
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 7)  # Setting default plot size

x_vals_1 = [1.92, 1.79, 1.96, 1.4, 1.61, 1.23, 1.43, 1.85, 2.07, 2.24, 2.11, 1.78, 2.21, 1.79, 1.33]
y_vals_1 = [2.5, 2.11, 2.19, 1.6, 2.83, 2.55, 2.33, 2.09, 2.32, 2.72, 2.05, 2.4, 2.55, 2.83, 1.58]

x_vals_2 = [3.63, 3.12, 3.21, 3.15, 3.56, 3.17, 3.05, 3.14, 2.87, 3.65, 2.82, 3.34, 3.7, 2.95, 3.15]
y_vals_2 = [3.1, 4.27, 4.03, 3.37, 3.22, 3.89, 3.27, 2.64, 3.09, 4.1, 3.61, 3.74, 3.71, 3.51, 2.9]


plt.scatter(x_vals_1, y_vals_1, label='Class 1', c='indigo', alpha=0.5)

plt.scatter(x_vals_2, y_vals_2, label='Class 2', c='blue', alpha=0.5)

plt.ylim(ymin=0, ymax=5)
plt.xlim(xmin=0, xmax=5)

plt.legend(loc=4)

plt.show()

In [ ]:
## Projecting all vectors to 2 dimensions using linear discriminant analysis (LDA) and viewing a scatter plot

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 7)  # Setting default plot size

lda = LDA(n_components=2) #2-dimensional LDA

y = cluster_classes

lda_transformed = pd.DataFrame(lda.fit_transform(X.toarray(), y))

lda_transformed['y'] = y

plt.scatter(lda_transformed[lda_transformed['y']==0][0], lda_transformed[lda_transformed['y']==0][1], label='0', c='blue', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==1][0], lda_transformed[lda_transformed['y']==1][1], label='1', c='green', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==2][0], lda_transformed[lda_transformed['y']==2][1], label='2', c='red', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==3][0], lda_transformed[lda_transformed['y']==3][1], label='3', c='violet', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==4][0], lda_transformed[lda_transformed['y']==4][1], label='4', c='orange', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==5][0], lda_transformed[lda_transformed['y']==5][1], label='5', c='magenta', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==6][0], lda_transformed[lda_transformed['y']==6][1], label='6', c='black', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==7][0], lda_transformed[lda_transformed['y']==7][1], label='7', c='indigo', alpha=0.4)

plt.legend(loc=2)

plt.show()

In [ ]:
## Projecting all vectors to 2 dimensions using principal component analysis (PCA) and viewing a scatter plot

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 7)  # Setting default plot size

pca = PCA(n_components=2) #2-dimensional LDA

pca_transformed = pca.fit_transform(X.toarray())

plt.scatter([pair[0] for pair in pca_transformed], [pair[1] for pair in pca_transformed], label='0', c='blue', alpha=0.4)

plt.legend(loc=2)

plt.show()

### *Clustering in topic space*

In [ ]:
%%capture
!pip3 install lda

In [ ]:
import lda

model = lda.LDA(n_topics=8, n_iter=1000, random_state=1)

model.fit(X)

In [ ]:
topic_features = model.doc_topic_

print(len(topic_features))

topic_features[:2]

In [ ]:
## Clustering documents using topic vectors (instead of vocabulary vectors, which we used above)

from sklearn.cluster import KMeans

kmeans_topic_classifier = KMeans(n_clusters=8, init='k-means++', n_init=15, max_iter=1000, algorithm='auto')
                                         ## ^ docuements to be grouped in 8 clusters 

topic_cluster_classes = kmeans_topic_classifier.fit_predict(topic_features) 

print(len(topic_cluster_classes))

topic_cluster_classes[:20]               ## Viewing first 20 cluster assignments

In [ ]:
## Write each document to a new text file, with each cluster of documents in its own directory

try: os.mkdir('/sharedfolder/nyt_topic_clusters_11-9-2017/')
except: pass

for i in range(len(documents_filtered)):
    
    out_dir = '/sharedfolder/nyt_topic_clusters_11-9-2017/Cluster_' + str(topic_cluster_classes[i])  ## Creating a directory pathname that
                                                                                         ## includes the assigned cluster number.
    try: os.mkdir(out_dir)  ## Creating the out_dir directory if it does not yet exist
    except: pass
    
    os.chdir(out_dir)
    
    out_filename = 'Document_' + str(i) + '.txt'    ## Creating a filename for the text file
    
    with open(out_filename, 'w') as file_out:
        file_out.write(document_list[i])            ## Writing text from original (non-preprocessed) document list


In [ ]:
## Projecting all topic vectors to 2 dimensions using linear discriminant analysis (LDA) and viewing a scatter plot

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 7)  # Setting default plot size

lda = LDA(n_components=2) #2-dimensional LDA

y = topic_cluster_classes

lda_transformed = pd.DataFrame(lda.fit_transform(topic_features, y))

lda_transformed['y'] = y

plt.scatter(lda_transformed[lda_transformed['y']==0][0], lda_transformed[lda_transformed['y']==0][1], label='0', c='blue', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==1][0], lda_transformed[lda_transformed['y']==1][1], label='1', c='green', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==2][0], lda_transformed[lda_transformed['y']==2][1], label='2', c='red', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==3][0], lda_transformed[lda_transformed['y']==3][1], label='3', c='violet', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==4][0], lda_transformed[lda_transformed['y']==4][1], label='4', c='yellow', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==5][0], lda_transformed[lda_transformed['y']==5][1], label='5', c='magenta', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==6][0], lda_transformed[lda_transformed['y']==6][1], label='6', c='black', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==7][0], lda_transformed[lda_transformed['y']==7][1], label='7', c='orange', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==8][0], lda_transformed[lda_transformed['y']==8][1], label='8', c='indigo', alpha=0.4)
plt.scatter(lda_transformed[lda_transformed['y']==9][0], lda_transformed[lda_transformed['y']==9][1], label='9', c='purple', alpha=0.4)

plt.legend(loc=2)

plt.show()

In [ ]:
## Viewing the most frequently used words in each topic cluster

import collections
from pprint import pprint


for topic_cluster_id in range(8):
    
    print('Topic cluster: ' + str(topic_cluster_id))    
    
    words = []
    
    for i in range(len(documents_filtered)):
        if topic_cluster_classes[i] == topic_cluster_id:
            words += documents_filtered[i].split(' ')

    counter = collections.Counter(words)
    pprint(counter.most_common()[:25])
    print()
